In [14]:
import math
import random
import numpy as np

def isqrt(n):
    '''Newtons method for finding the largest integer x for which x*x does
    not exceed n. This works for very large integer.'''
    x = n
    y = (x + 1) // 2
    while y < x:
        x = y
        y = (x + n // x) // 2
    return x


def is_prime2(n):
    '''Slow but accurate method for determining if a number is a prime.
    For very large primes, this gets really slow.'''
    if n < 2:
        return(False)
    if n==2:
        return(True)
    if n%2 == 0:
        return(False)
    else:
        for x in range(3,isqrt(n)+1,2):  # No need to test more than sqrt of n, turned integer, rounded up. Only odds
            if n%x == 0:
                return(False)
    return(True)

def is_prime(n):
    '''Slow but accurate method for determining if a number is a prime.
    For very large primes, this gets really slow.'''
    if n < 2:
        return(False)
    if n==2:
        return(True)
    if n%2 == 0:
        return(False)
    else: # No need to test more than sqrt of n, turned integer, rounded up. Only odds
        # for x in range(3,isqrt(n)+1,2):  # This is slower than the while loop.  
        x=3
        while x*x< n+1:
            if n%x == 0:
                return(False)
            x+=2
    return(True)


def primeSieve_slow(sieveSize):
    ''' Returns a list of prime numbers calculated using
    the Sieve of Eratosthenes algorithm.'''
    sieve = [True] * sieveSize
    sieve[0] = False # zero and one are not prime numbers
    sieve[1] = False
    # create the sieve
    for i in range(2, int(math.sqrt(sieveSize)) + 1): # You only need to sieve to sqrt(N)
        if sieve[i] == True:                          # Only need to check numbers that still may be primes.
            pointer = i * 2                           # This would be prime times 2, so not prime.
            while pointer < sieveSize:
                sieve[pointer] = False                # We thus mark it False
                pointer += i                          # And move to then next multiple of prime.
# compile the list of primes
#    primes = []
#    for i in range(sieveSize):
#        if sieve[i] == True:
#            primes.append(i)
    primes = [ i for i in range(sieveSize) if sieve[i]==True]  # This list comprehension version is faster.
    return primes

def primeSieve(sieveSize):
    ''' Returns a list of prime numbers calculated using
    the Sieve of Eratosthenes algorithm. Not practical for very
    large sieveSize. This version is more than twice the speed of the previous''' 
    # The following line does the same as the 5 lines above. Note how it treats sieveSize odd and even.
    sieve = [False,False,True,True]+[False,True]*(sieveSize//2 -2) + [False]*(sieveSize%2!=0)
    # create the sieve
    for i in range(3, int(math.sqrt(sieveSize)) + 1,2): # You only need to sieve every other up to sqrt(N)
        if sieve[i] == True:                          # Only need to check numbers that still may be primes.
#            for i in range(i*i,sieveSize,i):    # Mark every i-th value false, starting at i*i 
#                sieve[i]=False
#  This trick uses "slicing" of the list to quickly set particular values to False
            sieve[i*i::i]=[False]*((len(sieve)-i*i+i-1)//i) # len(sieve[i*i::i])
# compile the list of primes and return
    return [ i for i in range(sieveSize) if sieve[i]==True]  # This list comprehension version is faster.

Low_Primes = primeSieve(1000000)

def is_prime3(n):
    '''If you have a set of primes, then it is very quick to check.'''
    if n< Low_Primes[-1]:
        return( n in Low_Primes)
    else:
        is_prime(n)
        
def  break_into_primes(n):
    global Low_Primes
    if not 'Low_Primes' in globals():
        Low_Primes = primeSieve(1000000)

    if Low_Primes[-1] < n and n<1000000:
        Low_Primes = primeSieve(1000000)
    
    if is_prime(n):
        return( [(n,1)])

    out=[]
    
    t = n
    for p in Low_Primes:
        c = 0
        
        while t%p == 0:
            c+=1
            t=t/p
        if c>0:
            out.append( (p,c))
    
    if t == 1:
        return(out)
    else:
        out.append((t,1))
        if not is_prime(t):
            print("Result is incomplete",t)
        
    return(out)
        
# Primality Testing with the Rabin-Miller Algorithm
# http://inventwithpython.com/hacking (BSD Licensed)

def rabinMiller(num):
    '''Returns True if num is a probably a prime number. Since it uses trial
    functions, there is no guarantee. It is quite good though, and usually 
    the number is indeed prime if this returns true.'''
    s = num - 1
    t = 0
    while s % 2 == 0:
        # keep halving s while it is even (and use t
        # to count how many times we halve s)
        s = s // 2
        t += 1

    for trials in range(15): # try to falsify num's primality 15 times
        a = random.randrange(2, num - 1)
        v = pow(a, s, num)
        if v != 1: # this test does not apply if v is 1.
            i = 0
            while v != (num - 1):
                if i == t - 1:
                    return False
                else:
                    i = i + 1
                    v = (v ** 2) % num
    return True

def isProbablyPrime(num):
    global Low_Primes
    # Return True if num is a prime number. This function does a quicker
    # prime number check before calling rabinMiller().

    if (num < 2):
        return False # 0, 1, and negative numbers are not prime

    # About 1/3 of the time we can quickly determine if num is not prime
    # by dividing by the first few dozen prime numbers. This is quicker
    # than rabinMiller(), but unlike rabinMiller() is not guaranteed to
    # prove that a number is prime.
    if not 'Low_Primes' in globals():
        Low_Primes = primeSieve(1000000)

    if num in Low_Primes:
        return True

    # See if any of the low prime numbers can divide num
    for prime in Low_Primes:
        if (num % prime == 0):
            return False

    # If all else fails, call rabinMiller() to determine if num is a prime.
    return rabinMiller(num)

def is_prime4(n):
    '''If you have a set of primes, then it is very quick to check.'''
    if n< Low_Primes[-1]:
        return( n in Low_Primes)
    elif n<10000000:
        return(is_prime(n))
    else:
        return(isProbablyPrime(n))



def generateLargePrime(keysize=1024):
    # Return a random prime number of keysize bits in size.
    while True:
        num = random.randrange(2**(keysize-1), 2**(keysize))
        if isProbablyPrime(num):
            return num
        


In [15]:
def try_big_prime(n):
    t = 1
    for i in range(n):
        t *= primes[i]
    tt = t+1
    print("Try if ",tt," is a prime,",end="")
    if is_prime(tt):
        print("True")
    else:
        print("False",end="")
        print(break_into_primes(tt))
    tt = t-1
    print("Try if ",tt," is a prime,",end="")
    if is_prime(tt):
        print("True")
    else:
        print("False",end="")
        print(break_into_primes(tt))
       

In [3]:
is_prime(123456791)

True

In [4]:
pp=break_into_primes(152342346785)

In [16]:
is_prime4(pp[1][0])

True

In [6]:
print("Biggest known prime is: 2**57885161 - 1 ")
print("Not so big:",2**31-1)

Biggest known prime is: 2**57885161 - 1 
Not so big: 2147483647


In [26]:
for i in range(31,201):
    N=2**i-1
    if( isProbablyPrime(N)):
        print(i,rabinMiller(N))

31 True
61 True
89 True
107 True
127 True


In [37]:
%time p=primeSieve(100000000)

CPU times: user 11.9 s, sys: 1.69 s, total: 13.6 s
Wall time: 13.6 s


In [38]:
print(p[-10:])


[99999787, 99999821, 99999827, 99999839, 99999847, 99999931, 99999941, 99999959, 99999971, 99999989]


In [35]:
generateLargePrime(keysize=1024)

95370954047648968455972952284734382972879933549414318469400997045800069915885277652337486717277884894274105914740221714449392879832866579773740416161050616164268215240880177613662669876827674536680962242282168721034313136233966481067103633497392823506436116530024393960511393041666364194697042907991829012931

In [11]:
t=random.randrange(12345, 12355)

In [13]:
print("Find some big primes....")
for i in range(1024,2048):
    t=2**i - 5
    if isProbablyPrime(t):
        print(t,'-5')
    t=2**i + 5
    if isProbablyPrime(t):
        print(t,'+5)'
    t=2**i - 3
    if isProbablyPrime(t):
        print(t,'-3')
    t=2**i + 3
    if isProbablyPrime(t):
        print(t,'+3')


SyntaxError: invalid syntax (<ipython-input-13-c322b41e5870>, line 9)

In [96]:
sieve =[False,False]+[True]*209
sieve[2*2::2]=[False]*len(sieve[2*2::2])
sieve[3*3::3]=[False]*len(sieve[3*3::3])
sieve[5*5::5]=[False]*len(sieve[5*5::5])
#print(sieve)
i=8
print(len(sieve))
print(len(sieve[i*i::i]),(len(sieve)-i*i+i-1)//i)

211
19 19


# Finding Perfect Numbers
A perfect is a positive integer that is equal to the sum of its proper divisors. The smallest perfect number is 6, which is the sum of 1, 2, and 3. Other perfect numbers are 28, which is the sum of 1+2+4+7+14. Next are 496, and 8,128.

## Euclids method
(From Enceclopedia Brittanica)

A much faster method comes from Euclid, who stated:

    If as many numbers as we please beginning from a unit [1] be set out continuously in double proportion,
    until the sum of all becomes a prime, and if the sum multiplied into the last make some number, 
    the product will be perfect.

Here “double proportion” means that each number is twice the preceding number, as in 1, 2, 4, 8, …. For example, 1 + 2 + 4 = 7 is prime; therefore, 7 × 4 = 28 (“the sum multiplied into the last”) is a perfect number. Euclid’s formula forces any perfect number obtained from it to be even, and in the 18th century the Swiss mathematician Leonhard Euler showed that any even perfect number must be obtainable from Euclid’s formula. It is not known whether there are any odd perfect numbers.

Now note that the doubling numbers are all powers of two, so 2\*\*0 + 2\*\*1 + 2\*\*2 etc. 

In [4]:
import math

def divisorList(N):
    '''Quick divisor list creator.'''
    divs=[i for i in range(1,int(math.sqrt(N)+1)) if N%i==0 ]
    divs+=reversed([ int(N/i) for i in divs] )
    return(divs)

def divisorGenerator(n):
    '''This is needed to test directly from a perfect number, the brute force way.
    This function returns a list of perfect divisors for a number n.
    It is pretty fast, since it only brute force checks up to sqrt(n), and then lists the 
    complements. 
    Since this is a generator, it needs to be in a loop or a list()'''
    large_divisors = []
    for i in range(1, int(math.sqrt(n) + 1)):
        if n % i == 0:
            yield i
            if i*i != n:
                large_divisors.append(int(n / i))
    for divisor in reversed(large_divisors):
        yield divisor

def find_perfect_number(Max=1000000,Min=1):
    '''This is the slow brute force method to find perfect numbers. It is OK up to about 10**5
    but gets too slow for larger numbers.'''
    for ntry in range(Min,Max):
        pp=list(divisorGenerator(ntry))
        s= np.sum(pp[0:-1])
        if s == ntry:
            print("Perfect number {} from ".format(ntry),pp[0:-1])

def find_perfect_number_euclid(Max=60,Min=1):
    ''' Use Euclids method to find perfect numbers up to 2**Max, it is OK up to about Max=1000'''
    for ntry in range(Min,Max):
        # Using is_prime3, we switch to a very large prime number tester that is not 100% accurate!
        if is_prime3(sum([2**n for n in range(ntry)])):
            print("Perfect number {} from {} doublings".format(sum([2**n for n in range(ntry)])*2**(ntry-1),ntry))
        

In [ ]:
find_perfect_number(Max=100000,Min=1)

In [ ]:
%time find_perfect_number_euclid(Max=100,Min=1)

In [ ]:
2**62

In [6]:
Low_Primes = primeSieve(2**20)
len(Low_Primes)

82025

In [ ]:
is_prime(sum([2**n for n in range(17)]))

In [ ]:
4 // 3